<a href="https://colab.research.google.com/github/RochX/comp486-assignments/blob/main/assignment4/assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4
## Xavier Silva

## Part 1

### Question 1
Use `read_csv` in `pandas` to read the CSV (`adm_data.csv`) file and assign it to a variable and print it.

In [1]:
from pathlib import Path
import pandas as pd
import urllib.request

# download the data if it is not downloaded
if not Path("adm_data.csv").is_file():
  # here I download the data from my personal git repo for this class instead of using Google Drive
  url = "https://raw.githubusercontent.com/RochX/comp486-assignments/main/assignment2/adm_data.csv"
  urllib.request.urlretrieve(url, "adm_data.csv")

In [2]:
adm_data=pd.read_csv('adm_data.csv')
adm_data

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


### Question 2
Split the data into a training set and a test set.

In [3]:
import numpy as np

def shuffle_and_split_data(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [4]:
from sklearn.model_selection import train_test_split
np.random.seed(42)

adm_training_set, adm_test_set  = shuffle_and_split_data(adm_data, 0.15)
len(adm_training_set), len(adm_test_set)

(340, 60)

### Question 3
Using the training set, train a linear regression model to predict the chance of Admission to a university using `LinearRegression()` in Scikit-Learn, singular value
decomposition (SVD), Stochastic Gradient Descent, and Polynomial Regression with degree 2, then fill out the following table (show your full work). For each model:
1. What are the values of the model parameters ($\Theta$'s)?
2. Predict $\hat{y}$ for the new instance (`X_new`) where `X_new =[324, 112, 3, 2, 4.5, 8,1]`.
3. Calculate the score (using the test set).
4. Calculate the cost (`mean_squared_error`) (using the test set).
5. Write your observation about the results from the table.

*See the text cell with the table for all answers.*

In [5]:
# scale the training data
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler(feature_range=(0, 1)).set_output(transform="pandas")

adm_training_set_scaled = min_max_scaler.fit_transform(adm_training_set)
adm_training_set_scaled

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
124,0.309045,0.22,0.500000,0.75,0.375,0.500,0.535256,0.0,0.365079
294,0.736181,0.52,0.321429,0.25,0.375,0.250,0.487179,1.0,0.428571
255,0.638191,0.34,0.642857,0.75,0.750,0.875,0.503205,0.0,0.714286
78,0.193467,0.12,0.107143,0.25,0.500,0.250,0.237179,1.0,0.158730
101,0.251256,0.44,0.464286,0.25,0.375,0.500,0.423077,0.0,0.476190
...,...,...,...,...,...,...,...,...,...
71,0.175879,0.92,0.714286,1.00,1.000,1.000,0.948718,1.0,0.984127
106,0.263819,0.78,0.678571,0.75,0.875,0.875,0.762821,1.0,0.841270
270,0.675879,0.32,0.464286,0.25,0.375,0.500,0.455128,1.0,0.603175
348,0.871859,0.24,0.250000,0.00,0.250,0.250,0.144231,0.0,0.365079


We use the same scaler on the test data.
Notice that the minimums are not consistently `0` and the maximums are not consistently `1`.
This tells us we didn't actually use the `fit_transform` function on the test set and thus has the same scaling as the training set.

In [6]:
# use the same scaler on the test data
adm_test_set_scaled = min_max_scaler.transform(adm_test_set)
adm_test_set_scaled.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.450544,0.545000,0.554167,0.554167,0.627083,0.620833,0.595833,0.583333,0.600000
std,0.312760,0.250684,0.225637,0.284082,0.234127,0.233533,0.211935,0.497167,0.269973
min,-0.002513,0.120000,0.142857,0.250000,0.125000,0.125000,0.192308,0.000000,0.000000
25%,0.185930,0.360000,0.419643,0.250000,0.500000,0.500000,0.448718,0.000000,0.428571
50%,0.386935,0.580000,0.535714,0.500000,0.625000,0.625000,0.570513,1.000000,0.595238
75%,0.682789,0.700000,0.714286,0.750000,0.750000,0.750000,0.776442,1.000000,0.825397
max,0.992462,1.000000,1.000000,1.000000,1.000000,1.000000,0.967949,1.000000,0.984127


In [7]:
# split training set into features and labels

adm_training_features = adm_training_set_scaled.drop(["Serial No.", "Chance of Admit "], axis=1)
adm_training_labels = adm_training_set_scaled["Chance of Admit "]

In [8]:
# split test set into features and labels
adm_test_features = adm_test_set_scaled.drop(["Serial No.", "Chance of Admit "], axis=1)
adm_test_labels = adm_test_set_scaled["Chance of Admit "]

In [9]:
# set up the X_new variable with column names for each feature
X_new = [324, 112, 3, 2, 4.5, 8,1]
X_new = pd.DataFrame([X_new], columns=list(adm_training_features))
X_new

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,324,112,3,2,4.5,8,1


#### `LinearRegression()`
Train the model, get coefficients and intercepts.

In [10]:
# LinearRegression()
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(adm_training_features, adm_training_labels)
lin_reg.intercept_,lin_reg.coef_

(0.031208410934688624,
 array([0.14984229, 0.12218902, 0.0405335 , 0.00439134, 0.10905379,
        0.56415567, 0.03652352]))

Predict the chance of admission for the new instace.

In [11]:
lin_reg.predict(X_new)

array([67.43617394])

Calculate the score using the test set.

In [12]:
lin_reg.score(adm_test_features, adm_test_labels)

0.8102830881910719

Calculate the mean squared error.

In [13]:
from sklearn.metrics import mean_squared_error

adm_test_predictions = lin_reg.predict(adm_test_features)

lin_reg_rmse = mean_squared_error(adm_test_labels, adm_test_predictions, squared=False)
lin_reg_rmse

0.11660658504034582

#### Singular Value Decomposition

In [14]:
# Singular Value Decomposition

#### Stochastic Gradient Descent

In [15]:
# Stochastic Gradient Descent

#### Polynomial Regression with degree 2

In [16]:
# Polynomial Regression with degree 2

#### Table for question 3:
Note that $\theta_0$ is the intercept for linear regression.

All values for $\theta$ have been rounded to 3 decimal places.

| Model/Technique | Thetas $\Theta$'s | Predict `X_new` | Score | `mean_squared_error` |
| --- | --- | --- | --- | --- |
| Linear regression using Scikit-Learn | $\theta_0, \theta_1, \dots, \theta_7 = \{0.031, 0.150, 0.122, 0.041 , 0.004, 0.109, 0.564, 0.037\}$ | 67.44 | 0.81 | 0.12 |
| Singular Value Decomposition (SVD) |
| Stochastic Gradient Descent |
| Polynomial Regression with degree 2 |

### Question 4
Which model would you adopt and why?

### Question 5
Train a decision tree model with `max_depth=2`, then predict the chance of Admission of `X_new`

In [17]:
# TODO

## Part 2

### Question 1
Use `read_csv` in `pandas` to read the *training* set from the CSV ("Rock-Paper-
Scissor" training dataset) file and assign it to a variable and print it.

In [18]:
# TODO

### Question 2
Use `read_csv` in `pandas` to read the *test* set from the CSV ("Rock-Paper-Scissor"
test dataset) file and assign it to a variable and print it.

In [19]:
# TODO

### Question 3
Train a binary classifier to distinguish between just two classes, paper, and non-paper
using `LogisticRegression`, `LinearSVC (C = 0.01)`, and `LinearSVC (C = 100)`. For each model:
1. Predict $\hat{y}$ for some new instances (`X_new`) where (`X_new = Xtrain[88], Xtrain[2159], Xtrain[1346]`).
2. Find the probability that the model used to predict the class of an instance (if applicable).
3. Calculate the score (using the test set)
4. What are the `decision_function` values?
5. Write your observation about the results from the table.

In [20]:
# TODO code this...

Table for question 3:

| Model/Technique | Predict `X_new` | Probability | `decision_function` | Score
| --- | --- | --- | --- | --- |
| `LogisticRegression` |
| `LinearSVC (C = 0.01)` |
| `LinearSVC (C = 100)` |

### Question 4
How can you predict the class of an instance using:
1. Probability (if applicable)
2. `decision_function`

## Part 3


### Question 1
Use the same data (Rock-Paper-Scissor” dataset) in part 2 (training data and test data) to train multiclass classifiers using Softmax Regression, Polynomial Kernel SVM (degree=10, coef0=1, C=5), and Polynomial Kernel SVM with your suggested values.
For each model:
1. Predict $\hat{y}$ for some new instances (`X_new`) where (`X_new = Xtrain[88], Xtrain[2159], Xtrain[1346]`).
2. Find the probability that the model used to predict the class of an instance (if applicable).
3. Calculate the score (using the test set)
4. What are the `decision_function` values?
5. Write your observation about the results from the table.

In [21]:
# TODO

Table for question 1:

| Model/Technique | Predict `X_new` | Probability | `decision_function` | Score
| --- | --- | --- | --- | --- |
| `Softmax Regression` |
| `Polynomial Kernel SVM (degree=10, ceof0=1, C=5)` |
| `Polynomial Kernel SVM` **(your suggested values)** |

### Question 2
Train a grid search model to find the best combination to train SVM classifiers between `C` and `gamma` with `kernel = rbf` as follows:
```
para_grid = {'C': [0.1, 1, 10, 100, 1000],
            'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
            'kernel': ['rbf]}
```
1. What are the best values for the hyperparameters?
2. What is the best score?

In [22]:
# TODO